## LeNet
![](./Picture/LeNet.png)

1. 如果是灰度图像，则通道数为1；如果是彩色图像，通道数通常为3（RGB）。在LeNet模型中，in_channels=1 表示输入的图像为单通道的灰度图像。
2. out_channels=6 表示卷积层会使用6个不同的卷积核进行特征提取，因此输出的特征图将包含6个通道。
3. 卷积层+全连接层
4. `nn.Linear()` 函数是 PyTorch 中用于定义线性层的类。线性层（也称为全连接层）是神经网络中最基础的一种结构，它的作用是对输入进行线性变换，将输入数据映射到输出空间中。
   1. **定义权重和偏置参数：** 在初始化时，`nn.Linear()` 函数会随机初始化权重矩阵和偏置向量，这些参数会在训练过程中通过反向传播进行更新，以适应模型的训练目标。
   2. **进行线性变换：** 一旦定义了线性层，它可以对输入数据进行线性变换操作。其中，`input` 是输入数据（通常是一个特征向量或特征矩阵），`weight` 是权重矩阵，`bias` 是偏置向量，`output` 是线性层的输出。
   $$
   output = {input} \times {weight}^\top + {bias}
   $$
   3. **激活函数（可选）：** `nn.Linear()` 函数只进行线性变换，如果需要引入非线性，通常会在线性层后面添加一个激活函数（如 Sigmoid、ReLU 等）。
5. `output = self.fc(feature.view(img.shape[0], -1))`其中 img.shape[0] 是批量大小，-1 
6. 神经网络模型通常具有两种模式:通常，在训练过程中，模型会在每个批次的前向传播和反向传播时处于训练模式；而在验证集或测试集上进行评估时，模型会处于评估模式。
   1. 训练模式：在训练模式下，模型会考虑到 dropout 和 batch normalization 等操作的影响，因此这些操作会保持开启状态。
      1. 对于 dropout 操作，训练模式下会以一定的概率随机丢弃神经元，以减少模型过拟合。
      2. 对于 batch normalization 操作，训练模式下会根据当前的 batch 数据来计算均值和方差，并进行归一化操作。
   2. 评估模式：在评估模式下，模型不会考虑到 dropout 和 batch normalization 等操作的影响，因此这些操作会被关闭。
      1. 对于 dropout 操作，评估模式下会保留所有的神经元，以充分利用模型的所有信息。
      2. 对于 batch normalization 操作，评估模式下会使用模型在训练集上计算得到的全局均值和方差进行归一化，而不是根据当前的 batch 数据计算。
7. `.argmax(dim=1)`：对于每个样本，找到预测结果中具有最大值的索引，即预测的类别。
   1. 在多分类问题中，神经网络的输出通常是一个包含每个类别的得分的向量。为了得到最终的预测结果，我们需要确定哪个类别具有最高的得分，即找到预测结果中具有最大值的索引。这是因为，得分最高的类别往往被认为是模型最有信心的预测结果。
8. `== y.to(device)`：将真实标签 y 移动到与模型输出相同的设备上，并将其与预测的类别进行比较，得到一个布尔值的张量，表明预测是否正确。
9.  `.float().sum()`：将布尔值的张量转换为浮点数张量，并计算其中正确预测的样本数量。
10. `.cpu().item()`：将计算得到的结果移动到 CPU 上，并将其转换为 Python 中的标量值，以便进一步的处理。
11. 优化器更新步骤。
    1.  `optimizer.zero_grad()`：在优化器更新之前，首先调用 zero_grad() 方法来清除之前计算的梯度。这是因为PyTorch默认会在反向传播过程中累积梯度，而不是覆盖之前的梯度，因此在每个批次或每个训练步骤开始时，需要手动清除梯度，以避免梯度累积影响后续的优化步骤。
    2.  `backward()`：然后，通过调用 backward() 方法，计算损失函数 l 关于模型参数的梯度。这会自动计算损失函数对模型参数的偏导数，并将梯度存储在模型的各个参数的 .grad 属性中。
    3.  `optimizer.step()`：最后，调用 step() 方法来更新模型的参数。优化器会根据参数的梯度以及指定的优化算法来更新模型的参数值。具体而言，它会根据学习率和梯度更新规则来调整模型参数，以最小化损失函数。




In [1]:
import time
import torch
from torch import nn, optim

import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

In [3]:
BATCH_SIZE = 256

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=BATCH_SIZE)

def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device

    acc_sum, n = 0.0, 0
    with torch.no_grad():#关闭梯度计算，因为在评估模型时不需要计算梯度
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train
            else:
                if 'is_training' in net.__code__.co_varnanmes:
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item()
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            
            n += y.shape[0]
    return acc_sum / n

In [4]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):

    net = net.to(device)
    print("training on ", device)

    loss = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, batch_count, start = 0.0, 0.0, 0, 0, time.time()
        for X, y in train_iter:

            X = X.to(device)
            y = y.to(device)

            y_hat = net(X)

            l = loss(y_hat, y)
            
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            
            n += y.shape[0]
            batch_count += 1

        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / batch_count, train_acc_sum / n, test_acc, time.time() - start))


In [7]:
lr, num_epochs = 0.001, 5
lenet = LeNet()
optimizer = torch.optim.Adam(lenet.parameters(), lr=lr)
train(lenet, train_iter, test_iter, BATCH_SIZE, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.8377, train acc 0.326, test acc 0.579, time 7.7 sec
epoch 2, loss 0.9606, train acc 0.625, test acc 0.683, time 8.8 sec
epoch 3, loss 0.7973, train acc 0.705, test acc 0.719, time 8.9 sec
epoch 4, loss 0.7100, train acc 0.735, test acc 0.740, time 8.8 sec
epoch 5, loss 0.6517, train acc 0.752, test acc 0.751, time 8.8 sec
